# How Windows DLLs Work
* DLLs are implementations of shared libraries on the windows platform.
* DLLs have the same file format as executables
* There are two types of library files on windows, **static** (.lib) and **dynamic** (.dll). Static libs are linked at compile time whereas dynamic libs are linked at runtime.
* DLLs are loaded at startup, via a call to LoadLibrary
* The operating system will load the DLL into the executables memory space when the executable is loaded, or on demand at runtime.
* The DLL contains a special function "DLLMain" which is called on load to perform any initialisation the DLL requires.
* Multiple applications can use the same copy of a DLL, saving disk space.
* Within a PE file, there's an array of data structures, one per imported DLL. Each structure gives the name of the imported DLL and points to an array of function pointers (Import Address Table IAT).
* DLLs have a list of imports and exports, the imports include the name of the imported DLL.

### Refs
* http://stackoverflow.com/questions/124549/what-exactly-are-dll-files-and-how-do-they-work

## Linking an executable to a DLL
* There are two ways to link a DLL to an executable.
* **Implicit linking** (aka static load, load-time linking) - The executable using the DLL links to an import library (.lib) provided by the creator of the DLL. The OS loads the DLL when the exe is loaded. Code in the executable can call the DLL's exported functions just as if the functions were contained in the exe.
* **Explicit linking** - The executable makes calls to explicitly load and unload the DLL and access the DLL's exported functions. The code in the client executable calls exported functions through a function pointer.
* An executable can use the same DLL with either linking method (assuming a .lib file is provided?). Also, one exe can implicitly link a DLL and another can attach to it explicitly.
* The DLL loader ensures that any additional DLLs required by the module being loaded are also loaded.
* The import lib file for an implicitly linked DLL contains a stub for each function, which jumps to the code indicated by the IAT.
* This jmp instruction can be optomised away by giving a hint to the compiler (a declspec(dllimport)) that tells it that the function resides in another DLL and that the compiler should generated a direct call (to the function pointed at by the IAT) rather than the slower JMP instruction.

### Refs
* https://msdn.microsoft.com/en-us/library/9yd93633.aspx

## Implicit Linking
Executables require the following from the provider of the DLL
1. A header file, with everything properly declspeced.
* An import library (.lib file) to link with
* The actual DLL

When you implicitly link against an API, LoadLibrary and GetProcAddress like code still executes, but the loader does it all for you automatically.

### The Import Library
The import library for the DLL contains a stub for each function exported from the DLL.

### Refs
* https://msdn.microsoft.com/en-us/library/d14wsce5.aspx

## The search path used by windows to locate a DLL
For both implicit and explicit linking, windows searches for "known DLLs" e.g. Kernel32.dll, User32.dll, afterwards, it searches in the following sequence.

1. The directory where the executable is located.
*  The current directory
* The windows system directory (returned by GetSystemDirectory)
* The windows directory (returned by GetWindowsDirectory)
* The directories listed in the PATH envvar.

### Refs
* https://msdn.microsoft.com/en-us/library/7d83bc18.aspx

## Side By Side Assemblies
* Attempts to alleviate problems that arise from the use of DLLs
* Windows stores multiple versions of a DLL in the winsxs directory and loads them on demand. This reduces dependency problems for applications that include a side-by-side manifest.

